# Conv Network and Fashion MNIST

In [ ]:
import torch
import torchvision
from torch.utils import data

import numpy as np

from torch import nn
from torch import optim
from torch.nn import functional as F

In [ ]:
from torchvision.datasets.mnist import FashionMNIST
from torchvision.models.resnet import resnet18
from torchvision import transforms, utils

from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
transfs_tr = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.1307], [0.3081])
])

transfs_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.1307], [0.3081])
])

dset_tr = FashionMNIST(root='../data/fmnist', train=True, download=True, transform=transfs_tr)
dset_val = FashionMNIST(root='../data/fmnist', train=False, download=True, transform=transfs_val)
dataloader_tr = data.DataLoader(dset_tr, batch_size=64)
dataloader_val = data.DataLoader(dset_val, batch_size=64)

In [ ]:
dset_temp = FashionMNIST(root='../data/fmnist', train=True, download=True, transform=transforms.ToTensor())
dataloader_temp = data.DataLoader(dset_temp, batch_size=64)
batch_img = next(iter(dataloader_temp))[0]

grid = utils.make_grid(batch_img)

plt.figure(figsize=(10,10))
plt.imshow(grid.numpy().transpose(1,2,0))

### Labels
Each training and test example is assigned to one of the following labels:

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

https://github.com/zalandoresearch/fashion-mnist/blob/master/README.md

<table style="width:100%; table-layout:fixed;">
  <tr>
    <td><img width="150px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_no_strides.gif"></td>
    <td><img width="150px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/arbitrary_padding_no_strides.gif"></td>
    <td><img width="150px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/same_padding_no_strides.gif"></td>
    <td><img width="150px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/full_padding_no_strides.gif"></td>
  </tr>
  <tr>
    <td>No padding, no strides</td>
    <td>Arbitrary padding, no strides</td>
    <td>Half padding, no strides</td>
    <td>Full padding, no strides</td>
  </tr>
  <tr>
    <td><img width="150px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_strides.gif"></td>
    <td><img width="150px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/padding_strides.gif"></td>
    <td><img width="150px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/padding_strides_odd.gif"></td>
    <td></td>
  </tr>
  <tr>
    <td>No padding, strides</td>
    <td>Padding, strides</td>
    <td>Padding, strides (odd)</td>
    <td></td>
  </tr>
</table>



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.avg_pool2d(self.conv1(x), 2))
        x = F.relu(F.avg_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x
    
model = Net()

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
print(len(dataloader_tr))
for epoch in range(5):
    for i, (x, y) in enumerate(dataloader_tr):
        l = loss(model(x), y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('Epoch: {}, loss: {}'.format(epoch, l.item()))

In [ ]:
model.eval()
preds = []
ys = []
print(len(dataloader_val))
for i, (x, y) in enumerate(dataloader_val):
    preds.extend(model(x).max(1)[1].data.tolist())
    ys.extend(y.data)
    if not i % 30:
        print(i)

corrects = (np.array(preds) == np.array(ys))
print('Accuracy: {}'.format(corrects.mean()))


In [ ]:
from sklearn.metrics import confusion_matrix

plt.matshow(confusion_matrix(np.array(preds), np.array(ys)))

shirt get confused for t-shirt